![static_arm_0deg](static_arm_0deg.png)<br>
![static_arm_22deg](static_arm_22deg.png)<br>
![static_arm_45deg](static_arm_45deg.png)

In [16]:
import math

account_distribution = True

# Given constants
max_q = 56.21 # kPa
flap_h = 65 # mm
flap_w = 35 # mm
flap_A = flap_h * flap_w

# Conversion factor from kPa to N/mm^2
max_q_N = max_q * 0.001

# Applied force calculation
f_applied_max = max_q_N * flap_A
f_applied = f_applied_max * math.sin(45)


# Angle of the blue member with the horizontal
angle_degrees = 22
angle_radians = math.radians(angle_degrees)

# Friction coefficient (this is a rough estimate and should be determined experimentally or from literature)
friction_coefficient = 0.1  # Typical value for metal on metal contact

# Apply the friction coefficient to the force, viewing it as a loss from POV of servo
f_applied *= (1 + friction_coefficient)

# Recalculate force components considering friction
f_applied_x = f_applied * math.cos(angle_radians)
f_applied_y = f_applied * math.sin(angle_radians)

# The factor for determining where the equivalent applied force is located of the distributed the force along the member DC
distribution_factor = 0.5  # Assuming midpoint due to symmetry and results of FEA

# Adjust forces at Point C of member DC considering the distribution factor
f_applied_DC_cx = -f_applied_x * distribution_factor
f_applied_DC_cy = -f_applied_y * distribution_factor

# Since the system is symmetric, the forces at Point C of member CB will be the same but in the opposite direction
f_applied_CB_cx = -f_applied_DC_cx
f_applied_CB_cy = -f_applied_DC_cy

# The forces will be the same along the members since they are two-force members in this model
f_applied_CB_bx = f_applied_CB_cx
f_applied_CB_by = f_applied_CB_cy

f_applied_BA_bx = f_applied_CB_bx
f_applied_BA_by = f_applied_CB_by

# The force at Point A (attachment/contact point on the lead screw)
f_x_result = -f_applied_BA_bx
f_y_result = -f_applied_BA_by

print("Max applied force: ", f_applied, "N")
print("Resultant x force: ", f_x_result, "N")
print("Resultant y force: ", f_y_result, "N")


Max applied force:  119.69279100294217 N
Resultant x force:  -55.48861166298241 N
Resultant y force:  -22.418854347027445 N


![](https://blog.igus.in/wp-content/uploads/2021/10/efficiencycalculate.jpg)

In [17]:
def calculate_lead_screw_efficiency(lead, diameter, coefficient_of_friction, input_unit='imperial'):
    """
    Calculate the efficiency of a lead screw considering different materials for screw and nut.

    :param lead: The lead of the screw (distance the nut moves per screw revolution).
    :param diameter: The nominal diameter of the screw.
    :param coefficient_of_friction: The coefficient of friction between screw and nut materials.
    :param input_unit: 'imperial' or 'metric'.
    :return: Efficiency of the lead screw.
    """
    # Convert all inputs to imperial units for calculation
    if input_unit == 'metric':
        lead = lead * 0.0393701  # Convert lead to inches
        diameter = diameter * 0.0393701  # Convert diameter to inches

    # Calculate pitch diameter, assuming it's close to the nominal diameter
    pitch_diameter = diameter

    # Calculate the lead angle
    lead_angle = math.atan(lead / (math.pi * pitch_diameter))

    # Calculate angle of friction
    angle_of_friction = math.atan(coefficient_of_friction)

    # Calculate efficiency
    efficiency = math.tan(lead_angle) / math.tan(lead_angle + angle_of_friction)

    return efficiency  # Fractional efficiency

def calculate_pitch(threads_per_pitch, input_unit='imperial', output_unit='imperial'):
    """
    Calculate the pitch of a lead screw.

    :param threads_per_pitch: Threads per inch if imperial, pitch in mm if metric.
    :param input_unit: 'imperial' or 'metric'.
    :param output_unit: 'imperial' (threads per inch) or 'metric' (pitch in mm).
    :return: Pitch of the lead screw in the desired unit.
    """
    if input_unit == 'imperial':
        if output_unit == 'metric':
            # Convert threads per inch to pitch in mm (1 inch = 25.4 mm)
            pitch = 25.4 / threads_per_pitch
        else:
            pitch = threads_per_pitch  # Already in threads per inch
    else:  # input is metric
        if output_unit == 'imperial':
            # Convert pitch in mm to threads per inch (1 inch = 25.4 mm)
            pitch = 25.4 / threads_per_pitch
        else:
            pitch = threads_per_pitch  # Already in mm

    return pitch

# From : https://www.mcmaster.com/95075A340/ (PET Plastic Externally Threaded Precision Acme Nut)
coefficient_of_friction = 0.22  # Estimated for steel on PET plastic
# from: https://www.mcmaster.com/99030A985/ (1/4 20 1018 Carbon Steel Precision Acme Lead Screw)

'''
lead = 0.05 # imperial
diameter = 0.25 # imperial
threads_per_pitch = 20 # imperial
'''
# from: https://www.mcmaster.com/99030A981/ (1/4 16 1018 Carbon Steel Precision Acme Lead Screw)
lead = 0.063 # imperial
diameter = 0.25 # imperial
threads_per_pitch = 16 # imperial


lead_screw_pitch = calculate_pitch(threads_per_pitch, input_unit='imperial', output_unit='metric')
print("Lead screw pitch: ", lead_screw_pitch)
lead_screw_efficiency = calculate_lead_screw_efficiency(lead, diameter, coefficient_of_friction, 'imperial')
print("Lead screw efficiency: ", lead_screw_efficiency)

Lead screw pitch:  1.5875
Lead screw efficiency:  0.262474505690809


![Lead Screw Nut Torque](https://dev.nookindustries.com/media/3266/5ds.png)

In [18]:
def calculate_driving_torque(load, lead, efficiency):
    """
    Calculate the driving torque for a screw.
    
    Parameters:
    load (float): The load in pounds (lb).
    lead (float): The screw lead in inches/turn.
    efficiency (float): The efficiency of the screw.
    
    Returns:
    float: The driving torque in pound-inches.
    """
    return (load * lead) / (2 * math.pi * efficiency)

def calculate_holding_torque(load, lead, efficiency):
    """
    Calculate the holding torque for a screw.
    
    Parameters:
    load (float): The load in pounds (lb).
    lead (float): The screw lead in inches/turn.
    efficiency (float): The efficiency of the screw.
    
    Returns:
    float: The holding torque in pound-inches.
    """
    return (load * lead * efficiency) / (2 * math.pi)

def pound_inches_to_kg_cm(pound_inches):
    """
    Convert pound-inches to kilogram-centimeters.
    
    Parameters:
    pound_inches (float): The torque in pound-inches.
    
    Returns:
    float: The torque in kilogram-centimeters.
    """
    conversion_factor = 0.86796165979664
    return pound_inches * conversion_factor

def kg_cm_to_pound_inches(kg_cm):
    """
    Convert kilogram-centimeters to pound-inches.
    
    Parameters:
    kg_cm (float): The torque in kilogram-centimeters.
    
    Returns:
    float: The torque in pound-inches.
    """
    conversion_factor = 0.86796165979664
    return kg_cm / conversion_factor

# Example usage:
load = abs(f_y_result) * 3
load_lbs = load / 4.4482216

# Calculate torques
driving_torque = calculate_driving_torque(load_lbs, lead, lead_screw_efficiency)
holding_torque = calculate_holding_torque(load_lbs, lead, lead_screw_efficiency)
print(pound_inches_to_kg_cm(driving_torque), "kg-cm")
print(pound_inches_to_kg_cm(holding_torque), "kg-cm")

0.5013284506235318 kg-cm
0.034537953839794695 kg-cm
